In [1]:
import ipykernel
ipykernel.__version__

'6.15.2'

In [ ]:
!pip install numpy==1.23.1
!pip install pandas==1.4.3
!pip install matplotlib==3.3.2
!pip install seaborn==0.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 16.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 5.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
  Using cached matplotlib-3.3.2.tar.gz (37.9 MB)
  Preparing metadata (setup.py) ... done

# EDA:

The first step in any Data Science project is to understand the data that is given to us. There are few standard questions you can ask in order to get the understanding:
* What is the size of the dataset?
* What is the time range of the data?
* What is the meaning of each column and its values in the dataset? (can get this information from Data Dictionary)
* What is the distribution and unique values for each column? (univariate analysis)
* What is the distribution and unique values for some combinations of columns? (bivariate & multivariate analysis)

If you try to answer these questions with the help of data, then you would get a good understanding of the data which can be leveraged in **Feature Engineering** & **Model Building** part of the project.

# Contents:

I. [Loading the Data:](#Loading-the-Data:)

II. [Data Preprocessing:](#Data-Preprocessing:)

* [Extracting Location details:](#Extracting-Location-details:)

* [Cleaning all the text columns in the data:](#Cleaning-all-the-text-columns-in-the-data:)

* [Encoding Binary Features:](#Encoding-Binary-Features:)

* [Fixing Numerical Features:](#Fixing-Numerical-Features:)

III. [Univariate Analysis:](#Univariate-Analysis:)


*  [Distribution of Location and Sub-Area:](#Distribution-of-Location-and-Sub-Area:)

* [Distribution of Company and Township:](#Distribution-of-Company-and-Township:)

* [Distribution of Property Area and Price:](#Distribution-of-Property-Area-and-Price:)

    
    
IV. [Multivariate Analysis:](#Multivariate-Analysis:)

* [How Property area correlates with the price?](#How-Property-area-correlates-with-the-price?)

* [How each amenities correlates with the price?](#How-each-amenities-correlates-with-the-price?)

* [What are the 3 main amenities that the top 10% priced properties have?](#What-are-the-3-main-amenities-that-the-top-10%-priced-properties-have?)

V. [Treating Nulls and saving the data:](#Treating-Nulls-and-saving-the-data:)

## Loading the Data:

([Contents:](#Contents:))

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

import matplotlib.pyplot as plt
import seaborn as sns

import os
import time 
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the data
data= pd.read_excel('data/Pune Real Estate Data.xlsx')
print(data.shape)
data.head()

In [ ]:
df = data.copy()

In [ ]:
df

## Data Preprocessing:

#### Extracting Location details:

([Contents:](#Contents:))

In [ ]:
df['Location'].apply(lambda x: x.split(',')[2].lower().strip())

In [ ]:
# Extracting State and Country separately from the Location Column
df['City'] = df['Location'].apply(lambda x: x.split(',')[0].lower().strip())
df['State'] = df['Location'].apply(lambda x: x.split(',')[1].lower().strip())
df['Country'] = df['Location'].apply(lambda x: x.split(',')[2].lower().strip())
df.head(3)

#### Cleaning all the text columns in the data:

([Contents:](#Contents:))

##### Property Type:

In [ ]:
df['Propert Type'].value_counts()

In [ ]:
# Regex to match the numbers and create a separate column
numbers = re.compile(r"[-+]?(\d*\.\d+|\d+)") 
df['Property Type Cleaned'] = df['Propert Type'].apply(lambda x: numbers.findall(x)[0] 
                                                       if len(numbers.findall(x)) > 0 else 0)

In [ ]:
df['Property Type Cleaned'].value_counts()

##### Sub-Area:

In [ ]:
df['Sub-Area'].value_counts()

In [ ]:
# Number of categories before cleaning
len(df['Sub-Area'].value_counts())

In [ ]:
df['Sub-Area Cleaned'] = df['Sub-Area'].apply(lambda x: x.lower().strip())

In [ ]:
# Number of categories after cleaning
len(df['Sub-Area Cleaned'].value_counts())

##### Company Name:

In [ ]:
# Number of categories before cleaning
len(df['Company Name'].value_counts())

In [ ]:
df['Company Name Cleaned'] = df['Company Name'].apply(lambda x: x.lower().strip())

In [ ]:
len(df['Company Name Cleaned'].value_counts())

##### TownShip Name/ Society Name:

In [ ]:
# Number of categories before cleaning
len(df['TownShip Name/ Society Name'].value_counts())

In [ ]:
df['TownShip Name/ Society Name Cleaned'] = df['TownShip Name/ Society Name'].apply(lambda x: x.lower().strip())

In [ ]:
len(df['TownShip Name/ Society Name Cleaned'].value_counts())

##### Description:

In [ ]:
df['Description Cleaned'] = df['Description'].apply(lambda x: x.lower().strip())

In [ ]:
df.head(2)

#### Encoding Binary Features:

[Contents:](#Contents:)

In [ ]:
# Cleaning and encoding Binary Features
df['ClubHouse Cleaned'] = (df['ClubHouse'].apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))
df['School / University in Township Cleaned'] = (df['School / University in Township ']
                                                     .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))
df['Hospital in TownShip Cleaned'] = (df['Hospital in TownShip']
                                                     .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))
df['Mall in TownShip Cleaned'] = (df['Mall in TownShip']
                                                     .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))
df['Park / Jogging track Cleaned'] = (df['Park / Jogging track']
                                                     .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))
df['Swimming Pool Cleaned'] = (df['Swimming Pool']
                                                 .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))
df['Gym Cleaned'] = (df['Gym']
                             .apply(lambda x: x.lower().strip()).map({'yes':1, 'no':0}))

In [ ]:
df.tail()

#### Fixing Numerical Features:

[Contents:](#Contents:)

##### Property Area:

In [ ]:
numbers.findall(str("shot/tea"))

In [ ]:
numbers.findall(str("500/1000"))

In [ ]:
def avg_property_area(x):
    x = numbers.findall(x)
    if len(x) == 1:
        return np.float(x[0])
    elif len(x) == 2:
        return (np.float(x[0])+np.float(x[1]))/2
    else:
        return -99
    
numbers = re.compile(r"[-+]?(\d*\.\d+|\d+)")     
df['Property Area in Sq. Ft. Cleaned'] = df['Property Area in Sq. Ft.'].apply(lambda x: avg_property_area(str(x)))

In [ ]:
df[['Property Area in Sq. Ft.','Property Area in Sq. Ft. Cleaned']].sample(5)

##### Price:

In [ ]:
numbers = re.compile(r"[-+]?(\d*\.\d+|\d+)") 
df['Price in lakhs Cleaned'] = (df['Price in lakhs'].apply(lambda x: np.float(numbers.findall(str(x))[0]) 
                                                           if len(numbers.findall(str(x)))>0 else np.nan ))

In [ ]:
df[['Price in lakhs','Price in lakhs Cleaned']].sample(5)

In [ ]:
df.head()

In [ ]:
df.info()

## Univariate Analysis:

([Contents:](#Contents:))

In [ ]:
features = df.columns.tolist()[18:]
print("Total Number of Features: "+str(len(features)))

In [ ]:
df1 = df[features]
print(df1.shape)
df1.head(3)

#### Distribution of Location and Sub-Area:

([Contents:](#Contents:))

In [ ]:
# Location
print(f"Unique Number of Cities in the Data: {df1.City.nunique()}")

display(df1.groupby('City')['Company Name Cleaned'].count().reset_index())

plt.figure(figsize=(7,5))
sns.countplot(df1['City'])
plt.title("Distribution of data by City",size=15)
plt.show()

In [ ]:
# Sub-Area
print(f"Unique Number of Sub-Area in the Data: {df1['Sub-Area Cleaned'].nunique()}")

temp = df1.groupby('Sub-Area Cleaned')['City'].count().reset_index().sort_values(by='City',ascending=False)[:10]
display(temp)

plt.figure(figsize=(10,5))
sns.barplot(x=temp['Sub-Area Cleaned'],y=temp['City'])
plt.title("Distribution of data by Sub-Area",size=15)
plt.xticks(rotation=45)
plt.show()

#### Distribution of Company and Township:

([Contents:](#Contents:))

In [ ]:
# Company
print(f"Unique Number of Company in the Data: {df1['Company Name Cleaned'].nunique()}")

display(df1.groupby('Company Name Cleaned')['City'].count().sort_values(ascending=False).reset_index()[:10])

plt.figure(figsize=(20,5))
sns.countplot(df1['Company Name Cleaned'],
              order=df1.groupby('Company Name Cleaned')['City'].count().sort_values(ascending=False).index[:10])
plt.title("Distribution of data by Company",size=15)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Township
print(f"Unique Number of Township/Society in the Data: {df1['TownShip Name/ Society Name Cleaned'].nunique()}")

display(df1.groupby('TownShip Name/ Society Name Cleaned')['City'].count().sort_values(ascending=False)
          .reset_index()[:10])

plt.figure(figsize=(20,5))
sns.countplot(df1['TownShip Name/ Society Name Cleaned'],
              order=df1.groupby('TownShip Name/ Society Name Cleaned')['City'].count()
                      .sort_values(ascending=False).index[:10])
plt.title("Distribution of data by Township/Society",size=15)
plt.xticks(rotation=45)
plt.show()

#### Distribution of Property Area and Price:

([Contents:](#Contents:))

In [ ]:
display(df1[['Property Area in Sq. Ft. Cleaned']].describe())


In [ ]:
print("Summary Statistics of Property Area:")
display(df1[['Property Area in Sq. Ft. Cleaned']].describe())

plt.figure(figsize=(8,5))
plt.hist(df1['Property Area in Sq. Ft. Cleaned'],bins=20)
plt.title("Distribution of Property Area",size=15)
plt.xticks(rotation=45)
plt.show()

In [ ]:
print("Summary Statistics of Price:")
display(df1[['Price in lakhs Cleaned']].describe())

plt.figure(figsize=(8,5))
plt.hist(df1['Price in lakhs Cleaned'],bins=20)
plt.title("Distribution of Price",size=15)
plt.xticks(rotation=45)
plt.show()

In [ ]:
df1.head()

* How Property area correlates with the price?

* How each amenities correlates with the price?

* Create sum of all amenities and check how many Townships have all of it? And how the value of those properite vaires wrt others?

## Multivariate Analysis:

([Contents:](#Contents:))

Multivariate analysis helps us to understand how two or more columns are related to each other and its relationship with the target variable.

Based on the initial understanding of the different columns in the data (univariate analysis), we can create some hypothesis (if any) or try to ask more complex questions such as:
* How Property area correlates with the price?
* How each amenities correlates with the price?
* What are the 3 main amenities that the top 10% priced properties have?

This is not the exhaustive list but good to start with.

#### How Property area correlates with the price?

([Contents:](#Contents:))

In [ ]:
# Property Area vs Price
temp = df1[['Property Area in Sq. Ft. Cleaned','Price in lakhs Cleaned']]

display(temp.describe())

corr_matrix = temp.corr()
print(corr_matrix['Price in lakhs Cleaned'])

plt.figure(figsize=(8,5))
sns.scatterplot(temp['Property Area in Sq. Ft. Cleaned'],temp['Price in lakhs Cleaned'])
plt.title("Property Area vs Price",size=15)
plt.xticks(rotation=45)
plt.show()

#### How each amenities correlates with the price?

([Contents:](#Contents:))

In [ ]:
df1.head(2)

In [ ]:
feats_to_plot = df1.columns.tolist()[8:15]
print(len(feats_to_plot))

In [ ]:
fig,ax = plt.subplots(nrows=3,ncols=3,sharey=True,figsize=(20,15))
plt.suptitle("Distribution of Price by Ametnities",size=15)
for i,col in enumerate(feats_to_plot):
    sns.violinplot(x = df1[col],y=df1['Price in lakhs Cleaned'],ax=ax[i//3,i%3])

#### What are the 3 main amenities that the top 10% priced properties have?

([Contents:](#Contents:))

In [ ]:
# Filter for top 10% priced properties 
df1[['Price in lakhs Cleaned']].describe()

In [ ]:
# selecting threshold as 80 Lakhs
amenities_cols = df1.columns.tolist()[8:15]
amenities_cols

In [ ]:
df1[df1['Price in lakhs Cleaned']>=80][amenities_cols].sum().reset_index()

In [ ]:
temp = df1[df1['Price in lakhs Cleaned']>=80][amenities_cols].sum().reset_index()
temp.rename(columns={'index':'Amenities',0:'Popularity'},inplace=True)
temp = temp.sort_values(by='Popularity',ascending=False)
display(temp)

plt.figure(figsize=(15,5))
sns.barplot(x=temp['Amenities'],y=temp['Popularity'])
plt.title("Top Amenities in the Premium properties",size=15)
plt.xticks(rotation=60)
plt.show()

## Treating Nulls and saving the data:

([Contents:](#Contents:))

In [ ]:
df1.head()

In [ ]:
df1.isna().sum()

In [ ]:
df_final = df1.dropna()
df_final.isna().sum()

In [ ]:
# Saving the final dataframe with selected features as a csv file
df_final.to_csv('data/real_estate_selected.csv',index=False)